# Query Qdrant with Langchain

In [ ]:
! pip install langchain-qdrant langchain-openai langchain langchain-community

In [17]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain_qdrant import Qdrant
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_text_splitters import CharacterTextSplitter

## Init the embeddings

FastEmbedEmbeddings -> fetch files from Internet (Need to identify urls)
- embedding size is 384. 

OpenAIEmbeddings
- embedding size is 1536. 

Collection has been created for a specific embedding, so we can only use the same embeddings for the question

In [19]:
# FastEmbed
# embeddings = FastEmbedEmbeddings()

# OpenAIEmbeddings
from dotenv import load_dotenv
import os
import sys

load_dotenv()
base_url=os.getenv('AZURE_OPENAI_ENDPOINT')
print(f"API URL : {base_url}")


embeddings = AzureOpenAIEmbeddings(
    deployment="text-embedding-ada-002"
)

embeddings.embed_query("this is a test document")

API URL : https://oai-ygo74-switzerland.openai.azure.com/


[-0.012222585268318653,
 0.007210398558527231,
 -0.01481806393712759,
 -0.026444748044013977,
 -0.0034330501221120358,
 0.024392202496528625,
 -0.010818907991051674,
 -0.0042507583275437355,
 -0.01581123098731041,
 -0.024630561470985413,
 0.01402352936565876,
 0.027967605739831924,
 -0.019413121044635773,
 0.027755729854106903,
 -0.0012182858772575855,
 0.019214486703276634,
 0.015943653881549835,
 -0.011904771439731121,
 0.004005776718258858,
 -0.01550666056573391,
 0.0018572238041087985,
 0.000102420432085637,
 -0.017771083861589432,
 0.008236671797931194,
 -0.01848616451025009,
 -0.00664098234847188,
 0.01709572970867157,
 -0.033449891954660416,
 0.006339721381664276,
 -0.004969149827957153,
 0.0011495917569845915,
 -0.013414386659860611,
 -0.007276609539985657,
 -0.03511841595172882,
 -0.004840037785470486,
 0.0037641061935573816,
 -0.009838982485234737,
 -0.038667336106300354,
 0.024352476000785828,
 -0.017519481480121613,
 0.005551808048039675,
 0.007190535310655832,
 0.002600444

## Using en existing collection



In [35]:
collection_name = "web"
qdrant = Qdrant.from_existing_collection(
    embedding=embeddings,
    collection_name=collection_name,
    url="http://localhost:6333",
)

# Query the DB

In [36]:
query = "Quels sont les forfaits téléphoniques français avec itinérance aux États-Unis?"
found_docs = qdrant.max_marginal_relevance_search(query, k=2, fetch_k=10)

In [37]:
print(found_docs)
for doc in found_docs:
    print(doc)

[Document(page_content='', metadata={'_id': 'ff1001fd-0d0d-41e3-b38f-0ff324c8f859', '_collection_name': 'web'}), Document(page_content='', metadata={'_id': '7e740e81-faba-45ec-971f-b5e527b3c3f9', '_collection_name': 'web'})]
page_content='' metadata={'_id': 'ff1001fd-0d0d-41e3-b38f-0ff324c8f859', '_collection_name': 'web'}
page_content='' metadata={'_id': '7e740e81-faba-45ec-971f-b5e527b3c3f9', '_collection_name': 'web'}


In [38]:
retriever = qdrant.as_retriever()
retriever = qdrant.as_retriever(search_type="mmr")

query = "Quels sont les forfaits téléphoniques français avec itinérance aux États-Unis?"
retriever.invoke(query)[0]

Document(page_content='', metadata={'_id': 'ff1001fd-0d0d-41e3-b38f-0ff324c8f859', '_collection_name': 'web'})